# The network energy intensity of video streaming over Wi-Fi and 4G

**Authors:** David Mytton, Iain Staffell, Malte Jansen.

**Institution:** Centre for Environmental Policy, Imperial College London, London, SW7 1NE, UK.

**Correspondence:** <david@davidmytton.co.uk>.

## Summary

> Between 2010-2018 internet traffic grew ten-fold and is expected to double again by 2022. With video streaming making up 60% of that traffic, 65% on mobile, it is important to understand its energy consumption. Here we develop an approach to assessing network energy consumption by collecting internet routing observations from volunteers, provide updated figures for the energy consumption of different sections of the internet, then calculate the network energy intensity of video streaming. We estimate the 2019 network energy footprint of the UK’s 39 billion hours of video streaming at 4.2 TWh, or 1.3% of total electricity generation. We show that the network energy intensity of video streaming over Wi-Fi is 0.091 kWh/hour, compared to 0.207 kWh/hour over 4G mobile internet.

## This notebook

This notebook walks through the methodology and calculations as described in the accompanying manuscript.

#### Imports

In [1]:
%pip install -r requirements.txt

import pandas as pd

## Internet video traffic

We assume YouTube is representative of all streaming because it is the most watched streaming service, and so has the most data available. Using YouTube as a case study, we can calculate the total amount of time, data volume, and therefore energy consumption of video streaming in general.

We therefore start with an assessment of YouTube.

### YouTube watch time

YouTube does not publish statistics so we start with watching figures from the advertising industry<sup>1</sup>, Ofcom<sup>2</sup>, and Netflix<sup>3</sup>.

In [2]:
# Annual UK YouTube minutes (mobile devices)
# Monthly figure for Dec 2019 * 12 months for the whole of 2019
youtubeAnnualMinutesMobileDevices = 32099000000 * 12 # Source [1] (Pinpoint: pg 30)

# % watched YouTube (mobile devices)
youtubeWatchPercentageMobileDevices = 0.73 # 73% Source [2] (Pinpoint: Figure 4.10, pg116)

# % watched YouTube (mobile networks)
youtubeWatchPercentageMobileNetworks = 0.25 # 25% Source [3]

Using the total annual watch minutes, the total time watched on mobile devices and the total time watched on mobile networks, we can calculate the total time spent on Wi-Fi and non-mobile devices.

In [3]:
# Annual UK YouTube minutes (all networks, all devices)
youtubeAnnualMinutesAllNetworksAllDevices = youtubeAnnualMinutesMobileDevices / youtubeWatchPercentageMobileDevices
print(f'Annual UK YouTube minutes (all networks, all devices): {youtubeAnnualMinutesAllNetworksAllDevices:,.0f} mins')

print(f'Annual UK YouTube minutes (mobile devices): {youtubeAnnualMinutesMobileDevices:,.0f} mins')

# Annual UK YouTube minutes (mobile networks)
youtubeAnnualMinutesMobileNetworks = youtubeWatchPercentageMobileNetworks * youtubeAnnualMinutesAllNetworksAllDevices
print(f'Annual UK YouTube minutes (mobile networks): {youtubeAnnualMinutesMobileNetworks:,.0f} mins')

# Annual UK YouTube minutes (Wi-fi networks, all devices)
youtubeAnnualMinutesWifiAllDevices = youtubeAnnualMinutesAllNetworksAllDevices - youtubeAnnualMinutesMobileNetworks
print(f'Annual UK YouTube minutes (Wi-Fi networks, all devices): {youtubeAnnualMinutesWifiAllDevices:,.0f} mins')

Annual UK YouTube minutes (all networks, all devices): 527,654,794,521 mins
Annual UK YouTube minutes (mobile devices): 385,188,000,000 mins
Annual UK YouTube minutes (mobile networks): 131,913,698,630 mins
Annual UK YouTube minutes (Wi-Fi networks, all devices): 395,741,095,890 mins


### YouTube data volume

The amount of data transmitted during a single YouTube video streaming session varies based on factors such as device type, screen size, video resolution, framerate, bit rate, network speed and which formats the video was encoded into. 

For each video quality setting, we take the mean average of the range of values reported in an observational assessment of 1 hour of streaming produced for an Android website<sup>4</sup>. In this assessment, data volume was recorded using three separate tools: Android's built-in data monitoring, Google's Datally app and the GlassWire data monitoring app.

In [4]:
dataVolume = pd.DataFrame({
    480: { # 480p SD
        'low': 0.48,
        'high': 0.66
    },
    720: { # 720p HD
        'low': 1.2,
        'high': 2.7,

    },
    1080: { # 1080p FHD
        'low': 2.5,
        'high': 4.1,

    },
    1440: { # 1440p QHD
        'low': 2.7,
        'high': 8.1,

    },
    2160: { # 2160 4k
        'low': 5.5,
        'high': 23.0,

    },
})
dataVolume.mean()

480      0.57
720      1.95
1080     3.30
1440     5.40
2160    14.25
dtype: float64

### Video traffic

We need to know how much traffic is from YouTube and video in general for both mobile and non-mobile.

In [5]:
internetTrafficYouTube = 0.118 # 11.8% Source [5] (Pinpoint: pg12)
internetTrafficVideo = 0.60 # 60% Source [5] (Pinpoint: pg6)
mobileTrafficYouTube = 0.271 # 27.1% Source [6] (Pinpoint: pg9)
mobileTrafficVideo = 0.655 # 65.5% Source [6] (Pinpoint: pg5)

## Video streaming watch time & data volume

Using the figures above, we calculate the total annual time spent and associated data volume for YouTube.

We assume mobile traffic is all 4G and non-mobile traffic is all Wi-Fi because 97% of UK premises have access to decent fixed (at least 10 Mbit/s) and 4G (at least 2 Mbit/s) services, 91% of geographic areas of the UK are able to receive 4G data service from at least one operator, and 4G carries 90% of UK mobile data traffic<sup>7</sup>. 3G connectivity is excluded because it carries only a small proportion of data traffic.

We assume the default video quality is 720p High Definition (HD) because during the 2020 Coronavirus pandemic, the default was reduced to 480p Standard Definition (SD)<sup>8</sup>. The reference year for this assessment is 2019.

In [6]:
# 720p HD video quality
videoQuality = 720

### YouTube

In [7]:
# Calculate YouTube watch hours and data
youtube4GTime = youtubeAnnualMinutesMobileNetworks / 60 # Convert mins to hours
youtubeWiFiTime = youtubeAnnualMinutesWifiAllDevices / 60 # Convert mins to hours
youtubeTimeTotal = (youtubeAnnualMinutesMobileNetworks + youtubeAnnualMinutesWifiAllDevices) / 60 # Convert mins to hours
youtube4GData = youtube4GTime * dataVolume[videoQuality].mean()
youtubeWiFiData = youtubeWiFiTime * dataVolume[videoQuality].mean()
youtubeDataTotal = youtube4GData + youtubeWiFiData

youtube = pd.DataFrame({
    'Time': {
        '4G': youtube4GTime,
        'Wi-Fi': youtubeWiFiTime,
        'Total': youtubeTimeTotal,
    },
    'Data': {
        '4G': youtube4GData,
        'Wi-Fi': youtubeWiFiData,
        'Total': youtubeDataTotal,
    }
})

# Format nicely for display

# Convert to billion hours
youtube4GTimeBn = youtube['Time']['4G'] / 1000 / 1000 / 1000
youtubeWiFiTimeBn = youtube['Time']['Wi-Fi'] / 1000 / 1000 / 1000
youtubeTimeTotalBn = youtube['Time']['Total'] / 1000 / 1000 / 1000

# Convert from GB to EB
youtube4GDataEB = youtube['Data']['4G'] / 1000 / 1000 / 1000
youtubeWiFiDataEB = youtube['Data']['Wi-Fi'] / 1000 / 1000 / 1000
youtubeDataTotalEB = youtube['Data']['Total'] / 1000 / 1000 / 1000

youtubeDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{youtube4GTime:,.0f} ({youtube4GTimeBn:,.1f}bn) hours',
        'Wi-Fi': f'{youtubeWiFiTime:,.0f} ({youtubeWiFiTimeBn:,.1f}bn) hours',
        'Total': f'{youtubeTimeTotal:,.0f} ({youtubeTimeTotalBn:,.1f}bn) hours',

    },
    'Data': {
        '4G': f'{youtube4GData:,.0f} GB ({youtube4GDataEB:,.1f} EB)',
        'Wi-Fi': f'{youtubeWiFiData:,.0f} GB ({youtubeWiFiDataEB:,.1f} EB)',
        'Total': f'{youtubeDataTotal:,.0f} GB ({youtubeDataTotalEB:,.1f} EB)',

    }
})
youtubeDisplay

,Time,Data
4G,"2,198,561,644 (2.2bn) hours","4,287,195,205 GB (4.3 EB)"
Wi-Fi,"6,595,684,932 (6.6bn) hours","12,861,585,616 GB (12.9 EB)"
Total,"8,794,246,575 (8.8bn) hours","17,148,780,822 GB (17.1 EB)"


#### Validation by comparison

An Ofcom survey<sup>2</sup> reported the number of unique visitors to YouTube out of the UK population and how much time they spent on it per day in 2019. From this we can calculate the total viewing time and compare to our figure.

In [8]:
# YouTube daily visitors
ofcomYoutubeDailyVisitors = 41970000 # Source [2] (Pinpoint: Figure 4.6, pg111)

# YouTube average daily time spent (minutes)
ofcomYoutubeAverageMinutesDailyTimeSpent = 35 # Source [2] (Pinpoint: Figure 4.8, pg 114)

# Annual UK YouTube time spent (hours)
ofcomYoutubeAnnualTimeSpent = ((ofcomYoutubeAverageMinutesDailyTimeSpent * 365) * ofcomYoutubeDailyVisitors) / 60
ofcomYoutubeAnnualTimeSpentBn = ofcomYoutubeAnnualTimeSpent / 1000 / 1000 / 1000
print(f'Ofcom survey: {ofcomYoutubeAnnualTimeSpent:,.0f} ({ofcomYoutubeAnnualTimeSpentBn:,.1f}bn) hours')
print(f'Our calculation: {youtubeTimeTotal:,.0f} ({youtubeTimeTotalBn:,.1f}bn) hours')

# % difference
difference = ((youtubeTimeTotal - ofcomYoutubeAnnualTimeSpent) / youtubeTimeTotal) * 100
print(f'Difference: {difference:,.1f}%')

Ofcom survey: 8,936,112,500 (8.9bn) hours
Our calculation: 8,794,246,575 (8.8bn) hours
Difference: -1.6%


### All UK video streaming

Given that YouTube is a certain percentage of all internet traffic, and we know what percentage of traffic is video streaming, if we assume that YouTube is representative of all video streaming we can extrapolate for all UK video streaming.

In [9]:
# Extrapolate to all video streaming
allUKVideo4GData = (youtube4GData / mobileTrafficYouTube) * mobileTrafficVideo
allUKVideoWiFiData = (youtubeWiFiData / internetTrafficYouTube) * internetTrafficVideo
allUKVideo4GTime = allUKVideo4GData / dataVolume[videoQuality].mean()
allUKVideoWiFiTime = allUKVideoWiFiData / dataVolume[videoQuality].mean()

# Calculate totals
allUKVideoTimeTotal = allUKVideo4GTime + allUKVideoWiFiTime
allUKVideoDataTotal = allUKVideo4GData + allUKVideoWiFiData

allUKVideo = pd.DataFrame({
    'Time': {
        '4G': allUKVideo4GTime,
        'Wi-Fi': allUKVideoWiFiTime,
        'Total': allUKVideoTimeTotal,
    },
    'Data': {
        '4G': allUKVideo4GData,
        'Wi-Fi': allUKVideoWiFiData,
        'Total': allUKVideoDataTotal,
    }
})

# Format nicely for display

# Convert to billion hours
allUKVideo4GTimeBn = allUKVideo4GTime / 1000 / 1000 / 1000
allUKVideoWiFiTimeBn = allUKVideoWiFiTime / 1000 / 1000 / 1000
allUKVideoTimeTotalBn = allUKVideoTimeTotal / 1000 / 1000 / 1000

# Convert from GB to EB
allUKVideo4GDataEB = allUKVideo4GData / 1000 / 1000 / 1000
allUKVideoWiFiDataEB = allUKVideoWiFiData / 1000 / 1000 / 1000
allUKVideoDataTotalEB = allUKVideoDataTotal / 1000 / 1000 / 1000

allUKVideoDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{allUKVideo4GTime:,.0f} ({allUKVideo4GTimeBn:,.1f}bn) hours',
        'Wi-Fi': f'{allUKVideoWiFiTime:,.0f} ({allUKVideoWiFiTimeBn:,.1f}bn) hours',
        'Total': f'{allUKVideoTimeTotal:,.0f} ({allUKVideoTimeTotalBn:,.1f}bn) hours',

    },
    'Data': {
        '4G': f'{allUKVideo4GData:,.0f} GB ({allUKVideo4GDataEB:,.1f} EB)',
        'Wi-Fi': f'{allUKVideoWiFiData:,.0f} GB ({allUKVideoWiFiDataEB:,.1f} EB)',
        'Total': f'{allUKVideoDataTotal:,.0f} GB ({allUKVideoDataTotalEB:,.1f} EB)',

    }
})

allUKVideoDisplay

,Time,Data
4G,"5,313,866,704 (5.3bn) hours","10,362,040,072 GB (10.4 EB)"
Wi-Fi,"33,537,381,008 (33.5bn) hours","65,397,892,965 GB (65.4 EB)"
Total,"38,851,247,711 (38.9bn) hours","75,759,933,037 GB (75.8 EB)"


#### Validation by comparison

An Ofcom survey<sup>9</sup> reported the total number of video minutes watched per day per person and the proportion of that which was not broadcast content i.e. online video streaming vs broadcast TV. Based on the UK population<sup>10</sup>, we can calculate the total viewing time and compare to our figure.

In [10]:
ofcomTotalVideoDailyMinutes = 294 # Source [9] (Pinpoint: Figure 1.4, pg 16)
ofcomPercentageNonBroadcast = 0.31 # Source [9] (Pinpoint: Figure 1.4, pg 16)
ukPopulation = 66400000 # Source [10]

ofcomTotalVideoStreamingHours = (ofcomTotalVideoDailyMinutes * ofcomPercentageNonBroadcast * ukPopulation * 365) / 60
ofcomTotalVideoStreamingHoursBn = ofcomTotalVideoStreamingHours / 1000 / 1000 / 1000

print(f'Ofcom survey: {ofcomTotalVideoStreamingHours:,.0f} ({ofcomTotalVideoStreamingHoursBn:,.1f}bn) hours')
print(f'Our calculation: {allUKVideoTimeTotal:,.0f} ({allUKVideoTimeTotalBn:,.1f}bn) hours')

# % difference
difference = ((allUKVideoTimeTotal - ofcomTotalVideoStreamingHours) / allUKVideoTimeTotal) * 100
print(f'Difference: {difference:,.1f}%')

Ofcom survey: 36,814,484,000 (36.8bn) hours
Our calculation: 38,851,247,711 (38.9bn) hours
Difference: 5.2%


## References

1. UKOM (2019) Q4 2019 UK Digital Market Overview report. Available from: https://ukom.uk.net/uploads/files/news/ukom/174/UKOM_Digital_Marketing_Overview_December_2019_final.pdf

2. Ofcom (2020) Online Nation – 2020 report. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0027/196407/online-nation-2020-report.pdf

3. Solsman, J.E. (2018) Normally secretive Netflix inches back the curtain on how subscribers stream. 7 March 2018. CNET. Available from: https://www.cnet.com/news/netflix-shares-streaming-data-by-device-country-mobile-wi-fi-movies-tv/

4. Hindy, J. (2019) How much data does YouTube actually use? 30 June 2019. Android Authority. Available from: https://www.androidauthority.com/how-much-data-does-youtube-use-964560/

5. Sandvine (2020) The Mobile Internet Phenomena Report. Available from: https://www.sandvine.com/download-report-mobile-internet-phenomena-report-2020-sandvine

6. Sandvine (2019) The Global Internet Phenomena Report. Available from: https://www.sandvine.com/global-internet-phenomena-report-2019

7. Ofcom (2019) Connected Nations 2019. 20 December 2019. Ofcom. Available from: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2019/main-report

8. Chee, F.Y. (2020) YouTube, Amazon Prime forgo streaming quality to relieve European networks. Reuters. 20 March. Available from: https://uk.reuters.com/article/us-health-coronavirus-youtube-exclusive-idUKKBN2170OP

9. Ofcom (2019) Media Nations: UK 2019. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0019/160714/media-nations-2019-uk-report.pdf

10. Office for National Statistics (2019) Overview of the UK population. 23 August 2019. Available from: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/articles/overviewoftheukpopulation/august2019

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82ad941d-519a-4a94-b1a2-0a0958075b21' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>